# 第3节：校准与基准稳态 

## 表 3.1--模型核心参数校准

In [5]:
% =========================================================================
% == SCRIPT: Tab3_1.m
% == 版本: [v2.7 - 精简换行版]
% ==
% == 核心修改:
% ==   - [!!! 格式优化 !!!] 采纳您的建议，在 \midrule, \toprule,
% ==     \bottomrule 和 \multicolumn 这些本身就定义了行结构的命令后，
% ==     不再输出多余的换行符 "\\"。
% =========================================================================
clear; clc;
addpath(pwd);
fprintf('--- 启动 LaTeX 参数表直接生成脚本 (v2.7) ---\n');

%% --- 1. 加载数据并定义输出文件名 ---
data_filename = 'SS/data_for_transition.mat';
output_filename = 'tex/tab/Tab3.1.tex';

try
    load(data_filename, 'data_for_transition');
    cS = data_for_transition.cS;
    fprintf('   ✅ 已从 "%s" 加载参数结构体 cS。\n', data_filename);
catch ME
    error('无法加载转轨数据文件 "%s"。请先成功运行 main_run_ss.m。\n错误信息: %s', data_filename, ME.message);
end

%% --- 2. 构建参数数据 (Cell Array) ---
% 格式: { '命令或参数名', '符号', '数值', '来源', is_full_line_command }
% 最后一列是布尔值，用于标记是否为独占一行的命令
params_data = {
    '\midrule[0.75pt]', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{A. 偏好与生命周期参数}}\\', '', '', '', true;
    '相对风险规避系数', '$\sigma$', sprintf('%.1f', cS.sigma), '标准宏观文献', false;
    '主观贴现因子 (年化)', '$\beta_{annual}$', sprintf('%.3f', cS.beta^(1/cS.time_Step)), '校准目标', false;
    '人口稳态增长率 (年化)', '$n_{ss}$', sprintf('%.1f%%', cS.n_ss * 100), '长期人口预测', false;
    '存活率', '$\phi_a$', '时变', 'UN WPP (2024)', false;
    '模型期长度', '', sprintf('%d年', cS.time_Step), '模型设定', false;
    '退休年龄', '$a_{R}$', sprintf('%d岁', cS.ageRetire_orig), '模型设定', false;
    '\midrule', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{B. 生产技术参数}}\\', '', '', '', true;
    '技术进步率 (年化)', '$g_{A,ss}$', sprintf('%.1f%%', cS.g_A_ss * 100), '长期增长假设', false;
    '私人资本产出弹性', '$\alpha$', sprintf('%.3f', cS.alpha), 'Penn World Table', false;
    '公共资本产出弹性', '$\gamma$', sprintf('%.3f', cS.gamma), '校准目标', false;
    '私人资本折旧率 (年化)', '$\delta_k$', sprintf('%.1f%%', (1-(1-cS.ddk)^(1/cS.time_Step))*100), 'NIPA数据', false;
    '公共资本折旧率 (年化)', '$\delta_g$', sprintf('%.1f%%', (1-(1-cS.ddk_g)^(1/cS.time_Step))*100), '校准目标', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{C. 财政与PAYG养老金}}\\', '', '', '', true;
    '资本所得税率', '$\tau_k$', sprintf('%.1f%%', cS.tau_k * 100), '中国现行税制', false;
    '劳动所得税率', '$\tau_l$', sprintf('%.1f%%', cS.tau_l * 100), '中国现行税制', false;
    '消费税率', '$\tau_c$', sprintf('%.1f%%', cS.tau_c * 100), '中国现行税制', false;
    '政府消费 / GDP', '$G_c/Y$', sprintf('%.1f%%', cS.G_c_to_Y_ratio_ss * 100), '财政数据', false;
    '公共投资 / GDP', '$I_g/Y$', sprintf('%.1f%%', cS.I_g_to_Y_ratio_ss * 100), '财政数据', false;
    'PAYG缴费率', '$\theta_t$', '时变路径', '内生以平衡PAYG', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{D. 个人养老金 (PPS) 制度}}\\', '', '', '', true;
    'PPS最大缴费率 (占劳动收入)', '$pps_{max}$', sprintf('%.1f%%', cS.pps_max * 100), '中国个人养老金政策', false;
    'PPS提取率 (占PPS资产)', '', sprintf('%.1f%%', cS.pps_withdrawal_rate * 100), '模型简化假设', false;
    'PPS提取税率', '$\tau_{pps}$', sprintf('%.1f%%', cS.pps_tax_rate_withdrawal * 100), '中国个人养老金政策', false;
};
fprintf('   ✅ 已构建参数数据。\n');

%% --- 3. 准备输出目录并打开文件 ---
[output_dir, ~, ~] = fileparts(output_filename);
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('   已创建新目录: "%s"\n', output_dir);
end

fileID = fopen(output_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开最终输出文件: %s', output_filename); end
fprintf('   正在将完整的LaTeX表格代码写入 "%s"...\n', output_filename);

% --- 写入表格外层框架 ---
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{模型核心参数校准}\n');
fprintf(fileID, '\\label{tab:param_calibration}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\begin{tabular}{lccc}\n');
fprintf(fileID, '\\toprule\n');
fprintf(fileID, '参数名 & 参数符号 & 参数值 & 参数来源 \\\\\n');

% --- 循环写入表格主体内容 ---
for i = 1:size(params_data, 1)
    % --- 统一进行转义处理 ---
    col1 = strrep(params_data{i, 1}, '%', '\%');
    col2 = strrep(params_data{i, 2}, '%', '\%');
    col3 = strrep(params_data{i, 3}, '%', '\%');
    col4 = strrep(params_data{i, 4}, '%', '\%');
    is_command = params_data{i, 5};
    
    if is_command
        % 如果是整行命令，直接打印，不加换行符 "\\"
        fprintf(fileID, '%s\n', col1);
    else
        % 如果是普通的数据行，用 "&" 连接并换行
        fprintf(fileID, '%s & %s & %s & %s \\\\\n', col1, col2, col3, col4);
    end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n'); 
fprintf(fileID, '\\begin{tablenotes}[para]\n');
fprintf(fileID, '  \\item[注] 主观贴现因子、折旧率和人口增长率已根据模型周期长度 (%d年) 进行年化处理以便解读。\n', cS.time_Step);
fprintf(fileID, '  PAYG缴费率($\\theta_t$)和存活率($\\phi_a$)为时变路径，详情见正文。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}');

% --- 关闭文件 ---
fclose(fileID);

fprintf('   ✅ 成功生成语法正确的LaTeX表格文件: %s\n', output_filename);
fprintf('\n--- 脚本执行完毕 ---\n');

--- 启动 LaTeX 参数表生成脚本 ---
   ✅ 已从 "SS/data_for_transition.mat" 加载参数结构体 cS。
   ✅ 已根据模型设定构建参数表。
   ✅ 已调用 table2latex 生成临时文件 "temp_table_for_script.tex"。
   正在将表格内容写入最终框架并保存至 "Tab3.1.txt"...
   ✅ 成功生成最终的LaTeX表格文件: Tab3.1.txt

--- 脚本执行完毕 ---


## 表 3.2--稳态宏观经济结果比较

In [ ]:
% =========================================================================
% == SCRIPT: ss_comparison.m
% == 版本: [v1.0 - 稳态结果LaTeX表格生成器]
% ==
% == 目的:
% ==   1. 加载"无PPS" (ss_nopps) 和 "有PPS" (ss_pps) 的稳态结果。
% ==   2. 调用国民经济核算函数生成报告。
% ==   3. 选取核心宏观变量，比较两个稳态的差异。
% ==   4. 自动生成一个精美的、可直接用于论文的LaTeX表格 (Tab3.2.tex)。
% ==
% == 前置条件:
% ==   - 已成功运行 main_run_ss.m 并生成以下两个文件:
% ==     - SS/data_for_transition_nopps.mat
% ==     - SS/data_for_transition_pps.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== 稳态结果比较与LaTeX表格生成脚本 (v1.0) ===\n\n');

%% --- 1. 加载数据 ---
fprintf('--- 1. 加载两个情景的稳态数据 ---\n');

% --- 文件路径定义 ---
file_path_nopps = 'SS/data_for_transition_nopps.mat';
file_path_pps = 'SS/data_for_transition_pps.mat';
output_tex_filename = 'tex/tab/Tab3.2_ss_comparison.tex';

% --- 加载 "无PPS" (基准) 数据 ---
try
    data_nopps = load(file_path_nopps, 'data_for_transition');
    ss_nopps = data_nopps.data_for_transition.ss0; % 初始稳态
    cS_nopps = data_nopps.data_for_transition.cS;
    paramS_nopps = data_nopps.data_for_transition.paramS0;
    % 从dist0中恢复Z_norm
    dist0_abs_nopps = data_nopps.data_for_transition.dist0;
    Z_raw_nopps = sum(dist0_abs_nopps, [1 2 3]);
    Z_norm_nopps = Z_raw_nopps ./ sum(Z_raw_nopps);
    fprintf('   ✅ 已加载 "无PPS" (ss_nopps) 稳态数据。\n');
catch ME
    error('加载无PPS数据失败: %s', ME.message);
end

% --- 加载 "有PPS" (改革) 数据 ---
try
    data_pps = load(file_path_pps, 'data_for_transition');
    ss_pps = data_pps.data_for_transition.ssF; % 终期稳态
    cS_pps = data_pps.data_for_transition.cS;
    paramS_pps = data_pps.data_for_transition.paramSF;
    % 终期稳态使用其自身的Z分布
    Z_norm_pps = cS_pps.Z_path(:, end);
    fprintf('   ✅ 已加载 "有PPS" (ss_pps) 稳态数据。\n');
catch ME
    error('加载有PPS数据失败: %s', ME.message);
end

fprintf('   数据加载完成。\n');

%% --- 2. 生成国民经济核算报告 ---
fprintf('\n--- 2. 生成国民经济核算报告 ---\n');

% --- 生成报告结构体 (关闭命令行打印) ---
% 注意: 最后一个参数 is_bgp_ss 对 ss_nopps 是 false，对 ss_pps 是 true
report_nopps = SS.display_national_accounts(ss_nopps, cS_nopps, paramS_nopps, Z_norm_nopps, '', false, false);
report_pps = SS.display_national_accounts(ss_pps, cS_pps, paramS_pps, Z_norm_pps, '', false, true);

fprintf('   ✅ 核算报告已在内存中生成。\n');


%% --- 3. [核心] 定义比较变量并计算差异 ---
fprintf('\n--- 3. 定义比较变量并计算表格内容 ---\n');

% --- 定义要比较的变量和格式 (精简版) ---
% 格式: { 'LaTeX标签', '变量名', '格式', '变化类型', '单位' }
vars_to_compare = {
    '\multicolumn{4}{l}{\textbf{A. 宏观总量与价格}}', '', '', '', '';
    '有效人均产出 ($y$)', 'Y', '%.4f', 'abs', '';
    '有效人均私人资本 ($k_p$)', 'K_p', '%.4f', 'abs', '';
    '\quad 其中: 常规资本 ($k_k$)', 'K_k', '%.4f', 'abs', '';
    '\quad 其中: PPS 资本 ($k_{pps}$)', 'K_pps', '%.4f', 'abs', '';
    '私人资本/产出比 ($k_p/y$)', 'Kp_Y_ratio', '%.3f', 'diff', '';
    '年化真实利率 ($r$)', 'r_annual', '%.2f', 'diff', '\%';
    '有效工资率 ($w$)', 'w', '%.4f', 'abs', '';
    '\multicolumn{4}{l}{\textbf{B. 财政与养老金 (占产出Y的比例)}}', '', '', '', '';
    'PAYG缴费', 'PAYG_contrib', '%.2f', 'ratio_y', '\%';
    'PAYG福利', 'PAYG_benefit', '%.2f', 'ratio_y', '\%';
    'PPS缴费', 'PPS_contrib', '%.2f', 'ratio_y', '\%';
    'PPS提取', 'PPS_withdrawal', '%.2f', 'ratio_y', '\%';
};

% --- 初始化存储结果的 cell 数组 ---
table_data = cell(size(vars_to_compare, 1), 4); % {Label, Val_NoPPS, Val_PPS, Change}

% --- 循环处理每个变量 ---
for i = 1:size(vars_to_compare, 1)
    label = vars_to_compare{i, 1};
    var_name = vars_to_compare{i, 2};
    
    % 如果是子标题行
    if isempty(var_name)
        table_data{i, 1} = label;
        continue;
    end
    
    table_data{i, 1} = label;
    
    format_spec = vars_to_compare{i, 3};
    change_type = vars_to_compare{i, 4};
    unit = vars_to_compare{i, 5};
    
    % --- 提取数值 (增加稳健性检查) ---
    if isfield(report_nopps, var_name), val_nopps = report_nopps.(var_name); else, val_nopps = NaN; end
    if isfield(report_pps, var_name), val_pps = report_pps.(var_name); else, val_pps = NaN; end
    
    % --- 根据 ratio_y 类型预处理 ---
    if strcmp(change_type, 'ratio_y')
        if isfield(report_nopps, 'Y') && report_nopps.Y ~= 0, val_nopps = (val_nopps / report_nopps.Y) * 100; end
        if isfield(report_pps, 'Y') && report_pps.Y ~= 0, val_pps = (val_pps / report_pps.Y) * 100; end
    end
    
    % --- 格式化数值列 ---
    if isnan(val_nopps), table_data{i, 2} = '-'; else, table_data{i, 2} = [sprintf(format_spec, val_nopps), unit]; end
    if isnan(val_pps), table_data{i, 3} = '-'; else, table_data{i, 3} = [sprintf(format_spec, val_pps), unit]; end

    % --- 计算并格式化变化列 ---
    change_val_str = '-';
    if ~isnan(val_nopps) && ~isnan(val_pps)
        if strcmp(change_type, 'abs')
            if abs(val_nopps) > 1e-9, change = (val_pps / val_nopps - 1) * 100; else, change = NaN; end
            if ~isnan(change), change_val_str = sprintf('%.2f\\%%', change); end
        elseif strcmp(change_type, 'diff') || strcmp(change_type, 'ratio_y')
            change = val_pps - val_nopps;
            change_val_str = sprintf('%.2f pp', change);
        end
    end
    table_data{i, 4} = change_val_str;
end
fprintf('   ✅ 表格内容计算完成。\n');

%% --- 4. 生成 LaTeX 表格 ---
fprintf('\n--- 4. 生成 LaTeX 格式的稳态比较表格 ---\n');

% --- 准备输出目录和文件 ---
[output_dir, ~, ~] = fileparts(output_tex_filename);
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
fileID = fopen(output_tex_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTex输出文件。'); end

% --- 写入表格框架 ---
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '%%  此文件由 ss_comparison.m (v1.0) 自动生成\n');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{稳态宏观经济结果比较}\n');
fprintf(fileID, '\\label{tab:ss_comparison}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\begin{tabular}{lccc}\n');
fprintf(fileID, '\\toprule\n');
fprintf(fileID, '变量 & 期初稳态 & 期末稳态 & 变化 \\\\\n');
fprintf(fileID, '\\midrule\n');

% --- 循环写入表格内容 ---
for i = 1:size(table_data, 1)
    if isempty(vars_to_compare{i, 2}) % 如果是子标题
        fprintf(fileID, '%s \\\\\n', table_data{i, 1});
    else
        row_str = strjoin(table_data(i,:), ' & ');
        fprintf(fileID, '%s \\\\\n', row_str);
    end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\footnotesize\n');
fprintf(fileID, '  \\item[注] “无PPS情景”对应模型初始稳态 (ss0)，“有PPS情景”对应模型终期稳态 (ssF)。\n');
fprintf(fileID, '  除特别注明外，所有流量和存量均为单位有效劳动的标准化值。财政与养老金流量已表示为占各自稳态产出(Y)的百分比。\n');
fprintf(fileID, '  “变化”列中，“\\%%”表示相对变化率，“pp”表示百分点差异。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');

% --- 关闭文件 ---
fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_filename);
fprintf('\n--- 稳态比较脚本执行完毕 ---\n');

# 第4节：基准情形

## 图 4.1--人口冲击压力下的宏观经济转轨动态 ($\bar{g}=2\%$)

In [ ]:
% =========================================================================
% == SCRIPT: plot_figure_4_1_aging_impact_dashboard_v5_het.m
% ==
% == 版本: [v5.0 - 异质性模型适配版]
% ==
% == 目的:
% ==   - [核心] 将原有的绘图脚本适配至v20_heter的异质性家庭模型框架。
% ==   - [数据] 修改数据加载路径，以使用由异质性模型生成的结果文件。
% ==   - [变量] 为提高代码的精确性和可读性，对内部变量名进行了重构
% ==     (例如，明确区分k_hat_pc, y_hat_pc)，以准确反映其经济含义
% ==     (人均、去趋势的变量)。
% ==   - [逻辑] 计算人均量的核心逻辑得以保留，因为输入的results结构体
% ==     本身已包含在微观层面加总后的宏观总量。
% ==   - [美学] 维持原有的1x2灰度图布局和所有精修指令，确保图表风格
% ==     的连贯性。
% ==
% == 前置条件:
% ==   - 异质性模型转轨结果文件，例如: TRANS/TPI_results_het_nopps.mat
% =========================================================================

clear; close all;

fprintf('--- 启动图4.1绘制脚本 (v5.0 - 异质性模型适配版) ---\n');

%% --- 1. 用户设定 ---
year_max = 2300; % 控制图表展示的结束年份

%% --- 2. 定义文件路径与检查 ---
% [核心修改] 更新为异质性模型的结果文件
file_nopps_het = 'TRANS/TPI_results_het_nopps.mat'; 
output_dir = 'tex/fig';
output_filename = fullfile(output_dir, 'fig4.1_aging_impact_dashboard_het.png');

if ~exist(file_nopps_het, 'file'), error('找不到异质性模型结果文件: %s', file_nopps_het); end
if ~exist(output_dir, 'dir'), mkdir(output_dir); fprintf('   已创建输出目录: %s\n', output_dir); end

%% --- 3. [核心] 数据加载与重构 ---
fprintf('   正在加载并重构 "异质性PAYG-only" 情景数据...\n');

% --- 数据加载 ---
data = load(file_nopps_het);
cS = data.cS;

% --- 路径重构 ---
%   - total_pop_path: 经济中的总人口路径
%   - A_path:         技术水平路径 (TFP)
%   - AZ_path_pop:      用于将总量(level)转换为人均去趋势量(hat/per capita)的缩放因子
total_pop_path = sum(cS.Z_path_raw, 1);
AZ_path_pop = cS.A_path .* total_pop_path;

paths = struct();
% [变量重构] 明确变量含义: _hat 代表已除以A (去趋势), _pc 代表已除以总人口 (人均)
paths.k_hat_pc = data.results.K_p_path ./ AZ_path_pop;
paths.y_hat_pc = data.results.Y_path   ./ AZ_path_pop;
paths.r_path   = data.results.r_path;
paths.w_hat    = data.results.w_path   ./ cS.A_path; % w_path是w_hat*A，这里还原为w_hat

% 计算退休人口占比 (逻辑不变)
mass_retirees_path = sum(cS.Z_path_raw((cS.aR_new+1):end, :), 1);
paths.retiree_share = mass_retirees_path ./ total_pop_path;

% --- 时间轴截取 (逻辑不变) ---
T = cS.T_sim;
time_axis_full = (cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step));

time_mask = time_axis_full <= year_max;
time_axis = time_axis_full(time_mask);

% 对所有路径应用mask
fields = fieldnames(paths);
for i = 1:length(fields)
    if size(paths.(fields{i}), 2) == T
        paths.(fields{i}) = paths.(fields{i})(time_mask);
    end
end

fprintf('   ✅ 数据加载和重构完毕。\n');

%% --- 4. 绘图 ---
fprintf('   正在生成图表...\n');
fig = figure('Name', '老龄化冲击下的宏观经济动态(异质性模型)', 'Position', [461 603 730 319]);
tcl = tiledlayout(1, 2, 'Padding', 'compact', 'TileSpacing', 'normal');

% =========================================================================
% ==                      [绘图美学部分 - 保持不变]
% =========================================================================
% --- 1. 定义灰度图风格 ---
style_k    = {'k-', 'LineWidth', 2.5};     % 资本: 黑色实线
style_y    = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2.5}; % 产出: 深灰虚线
style_w    = {':', 'Color', [0.2 0.2 0.2], 'LineWidth', 2.5}; % 工资: 黑色点线
style_r    = {'k-', 'LineWidth', 2.5};     % 利率: 黑色实线
style_bg_line  = {'-.', 'Color', [0.8 0.8 0.8], 'LineWidth', 2}; % 背景线: 浅灰色虚线

color_left_axis = 'black';
color_right_axis = [0.5 0.5 0.5]; % 右轴: 中灰色

font_size_title = 14;
font_size_axis  = 11;
yy_lim_bar = [15 80]; % 固定Y轴范围 (百分比)

% --- 2. 准备X轴刻度 (确保包含首尾年份) ---
tick_start_year = ceil(time_axis(1)/50)*50;
xticks_vec = tick_start_year :30 : time_axis(end);
xticks_vec = unique([time_axis(1), xticks_vec]);

% --- 面板 1: 资本、产出与工资 ---
ax1 = nexttile;
hold(ax1, 'on');
set(ax1, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

% -- 右轴: 背景线 (退休人口占比) --
yyaxis(ax1, 'right');
p_bg_line = plot(ax1, time_axis, paths.retiree_share * 100, style_bg_line{:},'DisplayName',['退休人口占比' newline '(右轴, %)']);
set(ax1, 'YColor', color_right_axis);
ylim(ax1, yy_lim_bar);
ylabel_right_ax1 = get(ax1, 'YLabel');
if ~isempty(ylabel_right_ax1), set(ylabel_right_ax1, 'FontName', 'SimSun'); end

% -- 左轴: 三条主曲线 (人均、去趋势变量) --
yyaxis(ax1, 'left');
% [变量重构] 使用新的、更明确的变量名进行绘图
p_k = plot(ax1, time_axis, paths.k_hat_pc, style_k{:}, 'DisplayName', '人均有效私人资本');
p_y = plot(ax1, time_axis, paths.y_hat_pc, style_y{:}, 'DisplayName', '人均有效产出');
p_w = plot(ax1, time_axis, paths.w_hat,    style_w{:}, 'DisplayName', '有效工资率');
set(ax1, 'YColor', color_left_axis);
ylim([0,max(paths.k_hat_pc)*1.1]);
ylabel_left_ax1 = get(ax1, 'YLabel');
if ~isempty(ylabel_left_ax1), set(ylabel_left_ax1, 'FontName', 'SimSun'); end

% -- 图例和标签 --
title_obj_ax1 = title(ax1, '(a) 资本、产出与工资', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend_obj_ax1 = legend(ax1, [p_k, p_y, p_w, p_bg_line], 'Location', 'southeast', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');

% -- 通用设置 --
grid(ax1, 'on');
xlim(ax1, [time_axis(1), time_axis(end)]);
set(ax1, 'XTick', xticks_vec);
xtickangle(ax1, 40);
hold(ax1, 'off');

% --- 面板 2: 年化真实利率 ---
ax2 = nexttile;
hold(ax2, 'on');
set(ax2, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

% -- 右轴: 背景线 --
yyaxis(ax2, 'right');
p_bg_line_2 = plot(ax2, time_axis, paths.retiree_share * 100, style_bg_line{:});
set(ax2, 'YColor', color_right_axis);
ylim(ax2, yy_lim_bar);

% -- 左轴: 利率曲线 --
yyaxis(ax2, 'left');
r_annual = (1 + paths.r_path).^(1/cS.time_Step) - 1;
plot(ax2, time_axis, r_annual * 100, style_r{:});
set(ax2, 'YColor', color_left_axis);
ylim(ax2, 'padded');

% -- 图例和标签 --
legend_obj_ax2 = legend(ax2, p_bg_line_2, ['退休人口占比' newline '(右轴, %)'], ...
    'Location', 'northeast', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');
title_obj_ax2 = title(ax2, '(b) 年化真实利率 (%)', 'FontSize', font_size_title, 'FontName', 'SimSun');

% -- 通用设置 --
grid(ax2, 'on');
xlim(ax2, [time_axis(1), time_axis(end)]);
set(ax2, 'XTick', xticks_vec);
xtickangle(ax2, 40);
hold(ax2, 'off');

%% --- 5. 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_filename);
try
    exportgraphics(fig, output_filename, 'Resolution', 600);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

## 图 4.2--固定替代率承诺下的社保基金动态与财政压力 ($\bar{\rho}=40\%$)

In [ ]:
% =========================================================================
% == SCRIPT: plot_heter_pension.m
% ==
% == 版本: [v3.0 - 异质性模型聚合版]
% ==
% == 目的:
% ==   - 适配包含四类异质性家庭的OLG模型。
% ==   - 将各类型独立的PAYG收支流量进行加权汇总，分析总体现收现付缺口。
% ==   - 模拟一个统一的、外生的社保基金存量动态，以弥补该总缺口。
% ==   - 保持原有的1x2灰度图布局和专业美学风格。
% ==
% == 前置条件:
% ==   - 'TRANS/TPI_results_het_pps.mat' (或 _nopps.mat)
% ==   - 'TRANS/iter_results_het_pps.mat' (或 _nopps.mat)
% =========================================================================

clear; close all;
addpath(pwd);

fprintf('--- 启动异质性模型外生养老金基金模拟与绘图脚本 (v3.0) ---\n');

%% --- 1. 用户设定 ---
% 设定的外生目标养老金替代率 (相对于当期有效工资)
exo_rho = 0.40;
% 初始(t=1)时，社保基金存量占当年GDP的比率
initial_fund_to_gdp_ratio = 0.055;
% 图表展示的最终年份
year_max = 2100;

%% --- 2. 定义文件路径与加载数据 ---
% 根据您运行的是否有PPS的情景，选择正确的文件
use_pps_results = true; % <--- 在此切换 true/false

if use_pps_results
    file_tpi_results = 'TRANS/TPI_results_het_pps.mat';
    file_iter_results = 'TRANS/iter_results_het_pps.mat';
else
    file_tpi_results = 'TRANS/TPI_results_het_nopps.mat';
    file_iter_results = 'TRANS/iter_results_het_nopps.mat';
end

output_dir = 'tex/fig';
output_filename = fullfile(output_dir, 'fig4.2_exo_pension_fund_het.png');

if ~exist(file_tpi_results, 'file'), error('找不到TPI结果文件: %s', file_tpi_results); end
if ~exist(file_iter_results, 'file'), error('找不到TPI迭代文件: %s', file_iter_results); end
if ~exist(output_dir, 'dir'), mkdir(output_dir); fprintf('   已创建输出目录: %s\n', output_dir); end

fprintf('   正在加载异质性模型基准路径数据...\n');
data_tpi = load(file_tpi_results);
data_iter = load(file_iter_results);

cS = data_tpi.cS;
results = data_tpi.results;
% [核心修改] 加载的是分类型的养老金路径 [nH x T]
b_hat_path_h_iter = data_iter.b_hat_path_h_iter;
T = cS.T_sim;
nH = cS.nTypes;

fprintf('   ✅ 数据加载完毕。包含 %d 类家庭。\n', nH);

%% --- 3. [核心] 会计核算 (异质性聚合) ---
fprintf('   正在进行养老金总缺口与基金动态的会计核算 (聚合%d类家庭)...\n', nH);

% --- 准备宏观和人口路径 ---
r_path_level = results.r_path;
Y_path_level = results.Y_path;
w_hat_path = results.w_path ./ cS.A_path; % 标准化工资路径

% [核心修改] 计算分类型的退休人口绝对数量路径 [nH x T]
mass_retirees_total_path = sum(cS.Z_path_raw((cS.aR_new+1):end, :), 1);
mass_retirees_path_h = zeros(nH, T);
for h = 1:nH
    mass_retirees_path_h(h, :) = mass_retirees_total_path * cS.type_weights(h);
end

% --- 计算养老金福利差额 (承诺 vs. 内生平衡) ---
% 外生承诺的福利路径 (所有类型使用统一的替代率目标)
b_hat_path_exo = exo_rho * w_hat_path; % [1 x T]

% 福利差额路径 [nH x T]
b_hat_delta_h = repmat(b_hat_path_exo, nH, 1) - b_hat_path_h_iter;

% [核心修改] 计算需要由基金弥补的【总福利缺口】(level)
B_delta_total_level = sum(b_hat_delta_h .* cS.A_path .* mass_retirees_path_h, 1);

% --- 模拟外生基金存量动态 ---
P_level_path = zeros(1, T);
P_level_path(1) = initial_fund_to_gdp_ratio * Y_path_level(1);
fprintf('   初始(t=1)基金余额设定为: %.4f (基于GDP的%.2f%%)\n', P_level_path(1), initial_fund_to_gdp_ratio*100);

for t = 1:(T-1)
    % 基金积累方程: P(t+1) = P(t)*(1+r) - 缺口(t)
    % 缺口为正 (承诺>内生)，则基金减少；缺口为负，则基金增加。
    P_level_path(t+1) = P_level_path(t) * (1 + r_path_level(t)) - B_delta_total_level(t);
end

% --- 计算总的PAYG缴费和总的承诺福利 (用于绘图) ---
% 总缴费 = 总内生福利 (因为每个体系内部是平衡的)
Total_PAYG_Contrib_level = sum(b_hat_path_h_iter .* cS.A_path .* mass_retirees_path_h, 1);
% 总承诺福利
Total_Promised_Benefit_level = sum(repmat(b_hat_path_exo, nH, 1) .* cS.A_path .* mass_retirees_path_h, 1);

contrib_gdp_ratio = Total_PAYG_Contrib_level ./ Y_path_level;
benefit_gdp_ratio = Total_Promised_Benefit_level ./ Y_path_level;

fprintf('   ✅ 会计核算完成。\n');

%% --- 4. 准备绘图数据 ---
fprintf('   正在准备绘图数据...\n');
paths = struct();
paths.fund_level = P_level_path;
% [核心修改] 退休人口占比是全社会的总占比
paths.retiree_share = mass_retirees_total_path ./ sum(cS.Z_path_raw, 1);
paths.contrib_gdp = contrib_gdp_ratio;
paths.benefit_gdp = benefit_gdp_ratio;

time_axis_full = (cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step));
time_mask = time_axis_full <= year_max;
time_axis = time_axis_full(time_mask);

fields = fieldnames(paths);
for i = 1:length(fields)
    if size(paths.(fields{i}), 2) >= length(time_mask) % 确保路径长度足够
        paths.(fields{i}) = paths.(fields{i})(time_mask);
    end
end
fprintf('   图表将展示到 %d 年。\n', year_max);

%% --- 5. [核心] 绘图 ---
fprintf('   正在生成图表...\n');
fig = figure('Name', '外生社保基金动态与流量分析 (异质性聚合)', 'Position',[461 646 637 276]);
tcl = tiledlayout(1, 2, 'Padding', 'compact', 'TileSpacing', 'compact');

% --- 统一定义灰度风格和格式 ---
font_size_title = 12;
font_size_axis  = 10;
color_axis_main = 'black';
color_axis_bg = [0.5 0.5 0.5];

% ================= PANEL (a): 基金存量动态 =================
ax1 = nexttile;
hold(ax1, 'on');

style_fund = {'k-', 'LineWidth', 2};
style_retiree_share = {'--', 'Color', [0.5 0.5 0.5], 'LineWidth', 2};
style_zero_line = {'--', 'Color', [0.7 0.7 0.7], 'LineWidth', 1};

set(ax1, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

yyaxis(ax1, 'right');
p_retiree = plot(ax1, time_axis, paths.retiree_share * 100, style_retiree_share{:}, 'DisplayName', '退休人口占比 (右轴, %)');
set(ax1, 'YColor', color_axis_bg, 'YLim', [15 70]);
% YTick可以根据需要设置，例如: set(ax1, 'YTick', 15:5:70);

yyaxis(ax1, 'left');
p_fund = plot(ax1, time_axis, paths.fund_level, style_fund{:}, 'DisplayName', '社保基金余额 (左轴)');
yline(ax1, 0, style_zero_line{:});
set(ax1, 'YColor', color_axis_main);
ylabel(ax1, '基金余额 (万亿元)', 'FontName', 'SimSun');
% YLim可以根据数据动态设定以获得更好的可视化效果
fund_min = min(paths.fund_level);
fund_max = max(paths.fund_level);
ylim(ax1, [fund_min - 0.1*abs(fund_min), fund_max + 0.1*abs(fund_max)]);


title_obj1 = title(ax1, '(a) 基金存量动态', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend_obj1 = legend(ax1, [p_fund, p_retiree], 'Location', 'south', 'FontSize', 9, 'Box', 'off', 'FontName', 'SimSun');

grid(ax1, 'on'); box(ax1, 'on');
xlim(ax1, [time_axis(1), time_axis(end)]);
set(ax1, 'XTick', 2020:10:year_max);
xtickangle(ax1, 40);
hold(ax1, 'off');

% ================= PANEL (b): PAYG收支流量 =================
ax2 = nexttile;
hold(ax2, 'on');

style_contrib = {'k-', 'LineWidth', 2};
style_benefit = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2};
color_deficit_area = [0.8 0.8 0.8]; % 浅灰色区域

set(ax2, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);

deficit_gdp = paths.benefit_gdp - paths.contrib_gdp;
p_area = area(ax2, time_axis, deficit_gdp * 100, ...
    'FaceColor', color_deficit_area, 'FaceAlpha', 0.6, ...
    'EdgeColor', 'none', 'DisplayName', 'PAYG缺口');

p_contrib = plot(ax2, time_axis, paths.contrib_gdp * 100, style_contrib{:}, 'DisplayName', 'PAYG缴费收入');
p_benefit = plot(ax2, time_axis, paths.benefit_gdp * 100, style_benefit{:}, 'DisplayName', '承诺的福利支出');

ylim(ax2, [0, max(paths.benefit_gdp)*100*1.1]);
ylabel(ax2, '占GDP的百分比 (%)', 'FontName', 'SimSun');

title_obj2 = title(ax2, '(b) PAYG收支流量', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend_obj2 = legend(ax2, [p_contrib, p_benefit, p_area], 'Location', 'northwest', 'FontSize', 9, 'Box', 'off', 'FontName', 'SimSun');

grid(ax2, 'on'); box(ax2, 'on');
xlim(ax2, [time_axis(1), time_axis(end)]);
set(ax2, 'XTick', 2020:10:year_max);
xtickangle(ax2, 40);
hold(ax2, 'off');


fprintf('   ✅ 图表生成完毕。\n');

%% --- 6. 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_filename);
try
    exportgraphics(fig, output_filename, 'Resolution', 600);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

## 表 4.1--引入个人养老金制度对不同年龄组福利的影响 (截面分析)

## 图 4.3--引入个人养老金制度的各类型家庭内部福利变化分解

In [ ]:
% =========================================================================
% == SCRIPT: welfare_analysis.m
% == 版本: [v3.0 - 异质性家庭分解版]
% ==
% == 目的:
% ==   - 基于一个包含四类异质性家庭的模型，进行福利分析。
% ==   - 核心任务 1: 生成一张LaTeX表格，按家庭类型和总体平均，
% ==     展示在不同截面年份下，所有年龄组加权平均的福利变化(CV)。
% ==   - 核心任务 2: 生成一张2x2的图表，分类型展示青年、中年、
% ==     临近退休三个年龄组的CV随时间的变化路径。
% ==
% == 前置条件:
% ==   - 已运行 `main_run_trans.m` 两次，分别生成了带与不带PPS的转轨结果。
% ==   - TRANS/TPI_results_het_nopps.mat & TPI_results_het_pps.mat
% ==   - TRANS/iter_results_het_nopps.mat & iter_results_het_pps.mat
% ==   - SS/data_for_het_transition_nopps.mat & SS/data_for_het_transition_pps.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== OLG异质性模型福利分析脚本 (v3.0 - 类型分解版) ===\n\n');

%% --- 1. 设置与加载数据 ---
fprintf('--- 1. 设置与加载数据 ---\n');

% --- 文件路径定义 ---
file_path_nopps = 'TRANS/TPI_results_het_nopps.mat';
file_path_pps = 'TRANS/TPI_results_het_pps.mat';
iter_file_nopps = 'TRANS/iter_results_het_nopps.mat';
iter_file_pps = 'TRANS/iter_results_het_pps.mat';
ss_data_nopps_path = 'SS/data_for_het_transition_nopps.mat';
ss_data_pps_path = 'SS/data_for_het_transition_pps.mat';
output_tex_filename = 'tex/tab/tab4.1_welfare_analysis_by_type.tex';
output_fig_filename = 'tex/fig/fig4.3_welfare_analysis_detailed.png';

% --- 加载数据 ---
fprintf('   正在加载 "无PPS" (基准) 和 "有PPS" (改革) 情景数据...\n');
data_nopps = load(file_path_nopps);
iter_data_nopps = load(iter_file_nopps);
ss_data_nopps = load(ss_data_nopps_path);

data_pps = load(file_path_pps);
iter_data_pps = load(iter_file_pps);
ss_data_pps = load(ss_data_pps_path);
fprintf('   ✅ 数据加载完成。\n');


%% --- 2. [核心异质性修改] 重构价值函数路径 (分类型) ---
fprintf('\n--- 2. 重构价值函数路径 (分类型) ---\n');

nH = data_nopps.cS.nTypes;
Val_path_nopps_h = cell(nH, 1);
Val_path_pps_h = cell(nH, 1);

% --- a. 重构 "无PPS" 情景的价值函数路径 (循环所有类型) ---
fprintf('   正在为 "无PPS" 情景运行反向迭代...\n');
cS_nopps = data_nopps.cS;
paramSF_nopps = ss_data_nopps.data_for_transition.paramSF;
for h = 1:nH
    pathS_nopps_h = struct();
    pathS_nopps_h.r_path = data_nopps.results.r_path;
    pathS_nopps_h.w_hat_path = data_nopps.results.w_path ./ cS_nopps.A_path;
    pathS_nopps_h.b_hat_path = iter_data_nopps.b_hat_path_h_iter(h,:);
    pathS_nopps_h.tr_per_hh_hat_path = iter_data_nopps.TR_hat_pc_path_iter;
    A_path_ext_nopps = [cS_nopps.A_path, cS_nopps.A_path(end) * (1 + ((1+cS_nopps.g_A_ss)^cS_nopps.time_Step-1))];
    pathS_nopps_h.g_A_path = A_path_ext_nopps(2:end)./A_path_ext_nopps(1:end-1)-1;
    pathS_nopps_h.theta_path = cS_nopps.theta_path_h(h, :);
    
    cS_h_nopps = cS_nopps;
    cS_h_nopps.ageEffV_new = cS_nopps.ageEffV_new_h(:,h);

    valF_nopps_h = ss_data_nopps.data_for_transition.valF_h{h};
    polF_nopps_h = ss_data_nopps.data_for_transition.polF_h{h};
    
    [~, Val_path_nopps_h{h}] = household.backward_hh(pathS_nopps_h, cS_h_nopps, paramSF_nopps, valF_nopps_h, polF_nopps_h);
end

% --- b. 重构 "有PPS" 情景的价值函数路径 (循环所有类型) ---
fprintf('   正在为 "有PPS" 情景运行反向迭代...\n');
cS_pps = data_pps.cS;
paramSF_pps = ss_data_pps.data_for_transition.paramSF;
for h = 1:nH
    pathS_pps_h = struct();
    pathS_pps_h.r_path = data_pps.results.r_path;
    pathS_pps_h.w_hat_path = data_pps.results.w_path ./ cS_pps.A_path;
    pathS_pps_h.b_hat_path = iter_data_pps.b_hat_path_h_iter(h,:);
    pathS_pps_h.tr_per_hh_hat_path = iter_data_pps.TR_hat_pc_path_iter;
    A_path_ext_pps = [cS_pps.A_path, cS_pps.A_path(end) * (1 + ((1+cS_pps.g_A_ss)^cS_pps.time_Step-1))];
    pathS_pps_h.g_A_path = A_path_ext_pps(2:end)./A_path_ext_pps(1:end-1)-1;
    pathS_pps_h.theta_path = cS_pps.theta_path_h(h, :);

    cS_h_pps = cS_pps;
    cS_h_pps.ageEffV_new = cS_pps.ageEffV_new_h(:,h);

    valF_pps_h = ss_data_pps.data_for_transition.valF_h{h};
    polF_pps_h = ss_data_pps.data_for_transition.polF_h{h};

    [~, Val_path_pps_h{h}] = household.backward_hh(pathS_pps_h, cS_h_pps, paramSF_pps, valF_pps_h, polF_pps_h);
end

fprintf('   ✅ 价值函数路径重构完成。\n');


%% --- 3. [核心] 计算福利变化 (CV) ---
fprintf('\n--- 3. 计算各群体在特定年份的福利变化 ---\n');

% --- 定义分析维度 ---
assessment_years = [2023, 2030, 2040, 2050, 2070, 2100, 2150, 2200, 2250];
type_labels = {'高收入城镇职工'; '中低收入城镇职工'; '高收入居民'; '中低收入居民'};

% --- 定义绘图所需的代表性年龄组 (基于模型年龄索引 a_idx) ---
aR_idx = cS_nopps.aR_new;
age_groups_for_figure = {
    '青年 (25-29岁)', round((25 - cS_nopps.age1_orig)/cS_nopps.time_Step) + 1;
    '中年 (45-49岁)', round((45 - cS_nopps.age1_orig)/cS_nopps.time_Step) + 1;
    '临近退休 (60-64岁)', aR_idx
};
num_age_groups_fig = size(age_groups_for_figure, 1);

% --- 初始化结果存储 ---
num_years = length(assessment_years);
cv_table_data = zeros(nH, num_years); % 表格数据: 4种类型 x N个年份
cv_figure_data = zeros(num_age_groups_fig, num_years, nH); % 绘图数据: 3个年龄组 x N个年份 x 4种类型

% --- 核心计算循环 ---
sigma = cS_nopps.sigma;

for i_yr = 1:num_years
    year = assessment_years(i_yr);
    t_idx = round((year - cS_nopps.start_year) / cS_nopps.time_Step) + 1;
    
    fprintf('   正在评估年份: %d (t=%d)...\n', year, t_idx);
    
    if t_idx > cS_nopps.T_sim
        fprintf('      警告: 评估年份 %d (t=%d) 超出模拟范围，此列将留空。\n', year, t_idx);
        cv_table_data(:, i_yr) = NaN;
        cv_figure_data(:, i_yr, :) = NaN;
        continue;
    end
    
    for h = 1:nH
        % -- 为表格计算：所有年龄组的加权平均CV --
        total_mass_h_yr = 0;
        weighted_EV_nopps_h_yr = 0;
        weighted_EV_pps_h_yr = 0;

        for a_idx = 1:cS_nopps.aD_new
            dist_nopps_slice = data_nopps.final_Dist_path_h{h}(:, :, :, a_idx, t_idx);
            val_nopps_slice = Val_path_nopps_h{h}(:, :, :, a_idx, t_idx);
            mass_slice = sum(dist_nopps_slice, 'all');

            if mass_slice > 1e-12
                total_mass_h_yr = total_mass_h_yr + mass_slice;
                weighted_EV_nopps_h_yr = weighted_EV_nopps_h_yr + sum(val_nopps_slice .* dist_nopps_slice, 'all');
                
                dist_pps_slice = data_pps.final_Dist_path_h{h}(:, :, :, a_idx, t_idx);
                val_pps_slice = Val_path_pps_h{h}(:, :, :, a_idx, t_idx);
                weighted_EV_pps_h_yr = weighted_EV_pps_h_yr + sum(val_pps_slice .* dist_pps_slice, 'all');
            end
            
            % -- 为绘图计算：特定年龄组的CV --
            for i_age_grp = 1:num_age_groups_fig
                if a_idx == age_groups_for_figure{i_age_grp, 2}
                    EV_nopps_agegrp = sum(val_nopps_slice .* dist_nopps_slice, 'all') / max(1e-9, mass_slice);
                    
                    dist_pps_slice_agegrp = data_pps.final_Dist_path_h{h}(:, :, :, a_idx, t_idx);
                    val_pps_slice_agegrp = Val_path_pps_h{h}(:, :, :, a_idx, t_idx);
                    mass_pps_slice_agegrp = sum(dist_pps_slice_agegrp, 'all');
                    EV_pps_agegrp = sum(val_pps_slice_agegrp .* dist_pps_slice_agegrp, 'all') / max(1e-9, mass_pps_slice_agegrp);
                    
                    if EV_nopps_agegrp < 0 && EV_pps_agegrp < 0
                        cv_figure_data(i_age_grp, i_yr, h) = (EV_pps_agegrp / EV_nopps_agegrp)^(1 / (1 - sigma)) - 1;
                    else
                        cv_figure_data(i_age_grp, i_yr, h) = NaN;
                    end
                end
            end
        end

        EV_nopps_avg = weighted_EV_nopps_h_yr / max(1e-9, total_mass_h_yr);
        EV_pps_avg = weighted_EV_pps_h_yr / max(1e-9, total_mass_h_yr);
        
        if EV_nopps_avg < 0 && EV_pps_avg < 0
            cv_table_data(h, i_yr) = (EV_pps_avg / EV_nopps_avg)^(1 / (1 - sigma)) - 1;
        else
            cv_table_data(h, i_yr) = NaN;
        end
    end
end
fprintf('   ✅ 所有群体福利计算完成。\n');


%% --- 4. 生成 LaTeX 表格 ---
fprintf('\n--- 4. 生成 LaTeX 格式的福利分析表格 ---\n');

% --- 准备输出目录和文件 ---
[output_dir_tab, ~, ~] = fileparts(output_tex_filename);
if ~exist(output_dir_tab, 'dir'), mkdir(output_dir_tab); end
fileID = fopen(output_tex_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTex输出文件: %s', output_tex_filename); end

% --- 计算总体平均 ---
total_pop_by_year = sum(cS_nopps.Z_path_raw, 1);
cv_total_avg = zeros(1, num_years);
for i_yr = 1:num_years
    year = assessment_years(i_yr);
    t_idx = round((year - cS_nopps.start_year) / cS_nopps.time_Step) + 1;
    if t_idx > cS_nopps.T_sim, cv_total_avg(i_yr) = NaN; continue; end
    
    pop_weights_h = cS_nopps.type_weights;
    valid_mask = ~isnan(cv_table_data(:, i_yr));
    if any(valid_mask)
        cv_total_avg(i_yr) = sum(cv_table_data(valid_mask, i_yr) .* pop_weights_h(valid_mask)) / sum(pop_weights_h(valid_mask));
    else
        cv_total_avg(i_yr) = NaN;
    end
end


% --- 写入表格 ---
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '%%  此文件由 welfare_analysis.m (v3.0) 自动生成\n');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '\\begin{table}[htbp]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{引入个人养老金制度对不同类型家庭的福利影响 (截面分析)}\n');
fprintf(fileID, '\\label{tab4.1_welfare_analysis_by_type}\n');
fprintf(fileID, '\\begin{threeparttable}\n');

col_defs = ['l', repmat('c', 1, num_years)];
fprintf(fileID, '\\begin{tabular}{%s}\n', col_defs);
fprintf(fileID, '\\toprule\n');

fprintf(fileID, ' & \\multicolumn{%d}{c}{评估年份} \\\\\n', num_years);
fprintf(fileID, '\\cmidrule(lr){2-%d}\n', num_years + 1);

year_header_cells = arrayfun(@(y) num2str(y), assessment_years, 'UniformOutput', false);
year_header_line = strjoin(year_header_cells, ' & ');
fprintf(fileID, '家庭类型 & %s \\\\\n', year_header_line);
fprintf(fileID, '\\midrule\n');

% --- 循环写入各类型数据 ---
for h = 1:nH
    row_data_str = strjoin(arrayfun(@(cv) format_cv(cv), cv_table_data(h, :), 'UniformOutput', false), ' & ');
    fprintf(fileID, '%s & %s \\\\\n', type_labels{h}, row_data_str);
end

fprintf(fileID, '\\midrule\n');

% --- 写入总体平均 ---
total_avg_str = strjoin(arrayfun(@(cv) format_cv(cv), cv_total_avg, 'UniformOutput', false), ' & ');
fprintf(fileID, '\\textbf{总体平均} & %s \\\\\n', total_avg_str);

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\footnotesize\n');
fprintf(fileID, '  \\item[注] 表中数值为“有PPS情景”相对于“无PPS情景”的补偿变化(CV)。CV值为正，代表该群体福利因改革而提高。例如，+1.00\\%%表示该群体愿意放弃其剩余生命周期消费的1\\%%以换取改革。每格数值为该类型家庭在该年份下，对所有年龄组的福利变化进行人口加权平均得到的结果。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');
fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_filename);

%% --- 5. 生成 2x2 福利变化图 ---
fprintf('\n--- 5. 生成 2x2 福利变化分解图 ---\n');

% Figure尺寸和灰度风格定义
fig = figure('Name', '各类型家庭内部福利变化分解', 'Position', [100 497 632 498]);
% [核心修改] 设置TileSpacing和Padding为'compact'以最小化间距
tcl = tiledlayout(2, 2, 'Padding', 'compact', 'TileSpacing', 'compact');

% --- 定义灰度绘图风格 ---
style_young = {'-o', 'Color', [0 0 0], 'LineWidth', 2, 'MarkerSize', 5, 'MarkerFaceColor', [0.2 0.2 0.2]}; % 青年: 黑色实线 + 实心圆
style_middle = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2, 'Marker', 's', 'MarkerSize', 5};             % 中年: 深灰虚线 + 空心方块
style_preretire = {':d', 'Color', [0.6 0.6 0.6], 'LineWidth', 2, 'MarkerSize', 5};                        % 临近退休: 浅灰点线 + 空心菱形
styles = {style_young, style_middle, style_preretire};

subplot_titles = {'(a) 高收入城镇职工', '(b) 中低收入城镇职工', ...
                  '(c) 高收入居民', '(d) 中低收入居民'};

for h = 1:nH
    ax = nexttile;
    hold(ax, 'on');
    
    for i_age_grp = 1:num_age_groups_fig
        plot(ax, assessment_years, cv_figure_data(i_age_grp, :, h) * 100, ...
            styles{i_age_grp}{:}, 'DisplayName', age_groups_for_figure{i_age_grp, 1});
    end
    
    yline(ax, 0, 'k--', 'LineWidth', 1, 'HandleVisibility','off');
    
    title(ax, subplot_titles{h}, 'FontName', 'SimSun', 'FontSize', 14);
    xlabel(ax, '评估年份', 'FontName', 'SimSun');
    if mod(h, 2) == 1 % 只在左侧的图表显示Y轴标签
        ylabel(ax, '补偿变化 (CV %)', 'FontName', 'SimSun');
    end
    grid(ax, 'on');
    box(ax, 'on');
    xlim(ax, [assessment_years(1), assessment_years(end)]);
    
    % 只在第一个子图中显示图例
    if h == 1
        legend(ax, 'show', 'Location', 'best', 'FontName', 'SimSun');
    end
end

% --- 保存图像 ---
[output_dir_fig, ~, ~] = fileparts(output_fig_filename);
if ~exist(output_dir_fig, 'dir'), mkdir(output_dir_fig); end
fprintf('   正在保存图像至: %s\n', output_fig_filename);
try
    exportgraphics(fig, output_fig_filename, 'Resolution', 300);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 福利分析脚本执行完毕 ---\n');

%% --- [辅助函数] ---
function str_out = format_cv(cv_value)
    % 辅助函数: 将CV数值格式化为带正负号和百分号的字符串
    if isnan(cv_value)
        str_out = '-';
        return;
    end
    
    if cv_value >= 0
        sign_char = '+';
    else
        sign_char = '';
    end
    str_out = sprintf('%s%.2f\\%%', sign_char, cv_value * 100);
end

## 图 4.4--2050年各类型家庭生命周期储蓄与消费剖面

In [ ]:
% =========================================================================
% == SCRIPT: lifecycle_type.m
% == 版本: [v1.0]
% ==
% == 目的:
% ==   - 绘制不同制度下各类型家庭的生命周期储蓄与消费剖面图。
% ==   - 核心任务: 生成一张2x2的图表 (fig4.4_lifecycle_type)，每个子图
% ==     对应一类家庭，展示其在2050年横截面的生命周期行为。
% ==   - 每个子图包含四条线:
% ==     1. 储蓄率 (无PPS) = k_prime / 到手收入
% ==     2. 消费率 (无PPS) = c / 到手收入
% ==     3. 储蓄率 (有PPS) = k_prime / 到手收入
% ==     4. 消费率 (有PPS) = c / 到手收入
% ==
% == 前置条件:
% ==   - 已运行 `main_run_trans.m` 两次，分别生成了带与不带PPS的转轨结果。
% ==   - TRANS/TPI_results_het_nopps.mat
% ==   - TRANS/TPI_results_het_pps.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== OLG异质性模型生命周期剖面分析脚本 (v1.0) ===\n\n');

%% --- 1. 设置与加载数据 ---
fprintf('--- 1. 设置与加载数据 ---\n');

% --- 文件路径定义 ---
file_path_nopps = 'TRANS/TPI_results_het_nopps.mat';
file_path_pps = 'TRANS/TPI_results_het_pps.mat';
output_dir_fig = 'tex/fig/';
output_fig_filename = [output_dir_fig, 'fig4.4_lifecycle_type.png'];

% --- 检查并创建输出目录 ---
if ~exist(output_dir_fig, 'dir'), mkdir(output_dir_fig); end

% --- 加载数据 ---
fprintf('   正在加载 "无PPS" (基准) 和 "有PPS" (改革) 情景数据...\n');
if ~exist(file_path_nopps, 'file'), error('基准情景(nopps)的数据文件不存在: %s', file_path_nopps); end
if ~exist(file_path_pps, 'file'), error('改革情景(pps)的数据文件不存在: %s', file_path_pps); end
data_nopps = load(file_path_nopps);
data_pps = load(file_path_pps);
fprintf('   ✅ 数据加载完成。\n');


%% --- 2. [核心] 计算生命周期剖面 ---
fprintf('\n--- 2. 计算各类型家庭在2050年的生命周期剖面 ---\n');

% --- 统一定义与参数 ---
analysis_year = 2050;
type_labels = {'(a) 高收入城镇职工', '(b) 中低收入城镇职工', '(c) 高收入居民', '(d) 中低收入居民'};
scenarios = {'nopps', 'pps'};
loaded_data = {data_nopps, data_pps};

% --- 初始化结果存储容器 ---
% 维度: aD_new x nTypes x 2 (scenarios)
cS_ref = data_nopps.cS; % 使用一个cS作为参考
saving_rates_all = zeros(cS_ref.aD_new, cS_ref.nTypes, 2);
consumption_rates_all = zeros(cS_ref.aD_new, cS_ref.nTypes, 2);


% --- 遍历两种情景 (无PPS/有PPS) ---
for s = 1:2
    fprintf('   正在处理情景: %s ...\n', scenarios{s});
    
    % --- 为当前情景解包数据 ---
    data_scen = loaded_data{s};
    cS = data_scen.cS;
    final_Pol_path_h = data_scen.final_Pol_path_h;
    final_Dist_path_h = data_scen.final_Dist_path_h;

    % --- 定位到2050年对应的时间索引 t_idx ---
    t_idx = round((analysis_year - cS.start_year) / cS.time_Step) + 1;
    if t_idx > cS.T_sim
        warning('分析年份 %d 超出 %s 情景的模拟范围 T=%d', analysis_year, scenarios{s}, cS.T_sim);
        continue;
    end
    
    % --- 计算该时期的BGP增长因子 ---
    A_path_ext = [cS.A_path, cS.A_path(end) * (1 + ((1+cS.g_A_ss)^cS.time_Step-1))];
    g_A_period_path = A_path_ext(2:end)./A_path_ext(1:end-1)-1;
    growth_factor_bgp = 1 + g_A_period_path(t_idx);
    
    % --- 遍历四种家庭类型 ---
    for h = 1:cS.nTypes
        pol_t_h = final_Pol_path_h{h}{t_idx};
        dist_t_h_full = final_Dist_path_h{h}(:,:,:,:,t_idx);
        
        % --- 遍历所有年龄组 ---
        for ia = 1:cS.aD_new
            pol_age = pol_t_h(ia);
            dist_age = squeeze(dist_t_h_full(:, :, :, ia));
            
            total_mass_age = sum(dist_age, 'all');
            if total_mass_age < 1e-12, continue; end
            
            % 从策略中提取消费和储蓄决策 (hat量)
            c_mat = pol_age.c;
            k_prime_mat = pol_age.k_prime;
            
            % 根据预算约束反解出每个微观状态下的到手收入 (hat量)
            % 到手收入(hat) = (1+tau_c)*c(hat) + k_prime(hat) * (1+g_A)
            disposable_income_mat = (1 + cS.tau_c) * c_mat + k_prime_mat * growth_factor_bgp;
            
            % 聚合得到该年龄组的总量
            total_c_agg = sum(c_mat .* dist_age, 'all');
            total_k_prime_agg = sum(k_prime_mat .* dist_age, 'all');
            total_disposable_income_agg = sum(disposable_income_mat .* dist_age, 'all');
            
            % 计算平均比率
            if total_disposable_income_agg > 1e-9
                consumption_rates_all(ia, h, s) = total_c_agg / total_disposable_income_agg;
                saving_rates_all(ia, h, s) = total_k_prime_agg / total_disposable_income_agg;
            end
        end % 结束年龄循环
    end % 结束类型循环
end % 结束情景循环

fprintf('   ✅ 所有生命周期剖面计算完成。\n');


%% --- 3. 生成 2x2 图表 ---
fprintf('\n--- 3. 生成生命周期剖面图 (fig4.4) ---\n');

fig = figure('Name', '各类型家庭生命周期储蓄与消费剖面', 'Position', [100 495 706 500]);
tcl = tiledlayout(2, 2, 'Padding', 'compact', 'TileSpacing', 'compact');

% --- 定义灰度绘图风格 ---
% 无PPS: 较深的颜色, 实线(储蓄)+虚线(消费)
style_sr_nopps = {'-o', 'Color', [0.2 0.2 0.2], 'LineWidth', 1.5, 'MarkerSize', 3};
style_cr_nopps = {'--o', 'Color', [0.2 0.2 0.2], 'LineWidth', 1.5, 'MarkerSize', 3};
% 有PPS: 较浅的颜色, 实线(储蓄)+虚线(消费)
style_sr_pps = {'-s', 'Color', [0.6 0.6 0.6], 'LineWidth', 1.5, 'MarkerSize', 3};
style_cr_pps = {'--s', 'Color', [0.6 0.6 0.6], 'LineWidth', 1.5, 'MarkerSize', 3};

% --- 准备X轴: 物理年龄 ---
phys_ages = cS_ref.age1_orig:cS_ref.time_Step:(cS_ref.age1_orig + (cS_ref.aD_new-1)*cS_ref.time_Step);

for h = 1:cS_ref.nTypes
    ax = nexttile;
    hold(ax, 'on');
    
    % --- 绘制 无PPS 情景 ---
    p1 = plot(ax, phys_ages, saving_rates_all(:, h, 1), style_sr_nopps{:}, 'DisplayName', '储蓄率 (无PPS)');
    p2 = plot(ax, phys_ages, consumption_rates_all(:, h, 1), style_cr_nopps{:}, 'DisplayName', '消费率 (无PPS)');
    
    % --- 绘制 有PPS 情景 ---
    p3 = plot(ax, phys_ages, saving_rates_all(:, h, 2), style_sr_pps{:}, 'DisplayName', '储蓄率 (有PPS)');
    p4 = plot(ax, phys_ages, consumption_rates_all(:, h, 2), style_cr_pps{:}, 'DisplayName', '消费率 (有PPS)');
    
    % --- 标注退休年龄 ---
    retire_line = xline(ax, cS_ref.ageRetire_orig, 'k-.', 'LineWidth', 1, 'DisplayName', '退休年龄', 'HandleVisibility', 'off');
    
    % --- 图形格式化 ---
    title(ax, type_labels{h}, 'FontName', 'SimSun', 'FontSize', 14);
    xlabel(ax, '年龄', 'FontName', 'SimSun');
    if mod(h, 2) == 1 % 只在左侧图表显示Y轴标签
        ylabel(ax, '占当期到手收入比例', 'FontName', 'SimSun');
    end
    grid(ax, 'on');
    box(ax, 'on');
    xlim(ax, [cS_ref.age1_orig, cS_ref.ageLast_orig - cS_ref.time_Step]);
    ylim(ax, [0, 1.0]); % 比例通常在0到1之间

    % --- 在第一个子图中显示图例 ---
    if h == 1
        lgd = legend(ax, [p1, p2, p3, p4], 'Location', 'North', 'FontName', 'SimSun');
        lgd.NumColumns = 2;
    end
end

% --- 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_fig_filename);
try
    exportgraphics(fig, output_fig_filename, 'Resolution', 300);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 生命周期剖面分析脚本执行完毕 ---\n');

# 第五节：改革路径